In [26]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np

url = 'https://upload.wikimedia.org/wikipedia/commons/f/f2/Brachyscome_iberidifolia_Toucan_Tango_2zz.jpg'
response = requests.get(url)
image = Image.open(BytesIO(response.content))
image = image.convert('RGB')

image_array = np.array(image)
height = image_array.shape[0]
width = image_array.shape[1]
image_array_flat = image_array.flatten().reshape((
    height * width,
    3
))

view = image_array_flat.view((
    np.void,
    image_array_flat.dtype.itemsize * image_array_flat.shape[1]
))
num_colors = len(np.unique(view))

print(image_array_flat.shape, num_colors)

(665000, 3) 141862


In [37]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
labels = kmeans.fit_predict(image_array_flat)
centroids = np.round(kmeans.cluster_centers_).astype('uint8')

quantized_image_array_flat = centroids[labels]
quantized_image_array = quantized_image_array_flat.reshape(image_array.shape)

quantized_image = Image.fromarray(quantized_image_array, 'RGB')

quantized_image.save('Brachyscome_iberidifolia_Toucan_Tango_2zz_KMEANS3.png', 'PNG')
quantized_image.show()